In [2]:
D.<x> = PolynomialRing(QQ)
R = PolynomialRing(QQ,'x')
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
Scplx= PowerSeriesRing(CC,'x',default_prec=300)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m): 
    f=J(n,m)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m):
    return xjNewStep(n-1,m)
def xjNewCoefficient(n,m):
    if n>0:return (xjNew(n,m)-xjNew(n-1,m))/x^n
    if n==0:return 1
def xjNewTest(n,m):
    return D(xjNewStep(n-1,m))
def j(n,m):return xjNew(n,m)/x

def H4(n,m): # this is the definition in the published interpolations paper (2021):
    # def. 8.4
    jay=J(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-1)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()
    #if argument is an S type then 'polynomial' takes no argument.

def H4strike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=H4(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer


def H6(n,m):
    def base6(n,m):
        jay=J(n+1,m)
        num=(x*derivative(jay,x))^m # chain rule; variable is tau not x.
        den = expand(jay^(m-1)*(jay-1))
        return L((-1)^m*(num/den)).O(n+1)
    return S(base6(n,m)^(1/(m-2))).polynomial()
     #if argument is an S type then 'polynomial' takes no argument.

def H6strike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=H6(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer

def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def nieburTau(n):
# The contortions below sidestep mysterious (to me) error messages...
# nieburTau is an algorith for Ramanujan''s tau from the paper
# "A formula for Ramanujan's tau-function" by Douglas Niebur.
# (He spells tau with the Greek letter.)
    def step1(n,k):
        return 35*k^4-52*k^3*n+18*k^2*n^2
    def step2(n,k):
        return sigma(k)*sigma(n-k)
    def step3(n):
        ans=0
        for k in [1..n-1]:
            ans=ans+step1(n,k)*step2(n,k)
        return 24*ans
    return n^4*sigma(n)-step3(n)

def deltaDagger(n,m):
    def ddstep1(n,m):
        return D(S(expand((H4(n+2,m)^3)))).truncate(n+1)
    def ddstep2(n,m):
        return D(S((H6(n+2,m)^2))).truncate(n+1)
    return ((ddstep1(n,m)-ddstep2(n,m))/1728)

def deltaDaggerStrike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=deltaDagger(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [1..polydegree])
    return answer

def deltaDagger2(n,m):
    def ddstep1(n,m):
        return D(S(expand((H4(n+2,m)^3)))).truncate(n+1)
    def ddstep2(n,m):
        return D(S((H6(n+2,m)^2))).truncate(n+1)
    return (ddstep1(n,m)-ddstep2(n,m))

# def deltaDaggerStrike(n,m):
 #   return S(((H4strike(n,3)^3-H6strike(n,3)^2)/1728)).truncate(n+1)
    
def deltaDagger2Strike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=deltaDagger2(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [1..polydegree])
    return answer

def first(list):return list[0]
def last(list):return list[-1]

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr



print("ok")

ok


In [2]:
data=[]
import time
start=time.time()/60
for m in [3..1203]:
    poly=H4(200,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/24sept22no1.txt','wb') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[3, 200, 0.27416178584098816]
[4, 200, 0.4724309481680393]
[5, 200, 0.8450320325791836]
[6, 199, 1.0186402350664139]
[7, 200, 1.4207725822925568]
[8, 200, 1.7371144481003284]
[9, 200, 2.1730619184672832]
[10, 200, 2.452136803418398]
[11, 200, 2.9029116332530975]
[12, 200, 3.285788781940937]
[13, 200, 3.7538143694400787]
[14, 200, 4.057925716042519]
[15, 200, 4.588935501873493]
[16, 200, 4.955913379788399]
[17, 200, 5.455510836094618]
[18, 200, 5.793895915150642]
[19, 200, 6.281520381569862]
[20, 200, 6.718764949589968]
[21, 200, 7.264707986265421]
[22, 200, 7.592765718698502]
[23, 200, 8.091646380722523]
[24, 200, 8.550945565104485]
[25, 200, 9.102406837046146]
[26, 200, 9.475031901150942]
[27, 200, 10.025452714413404]
[28, 200, 10.49638594686985]
[29, 200, 11.017314501106739]
[30, 200, 11.448775451630354]
[31, 200, 11.988834667950869]
[32, 200, 12.422229766845703]
[33, 200, 13.02582198008895]
[34, 200, 13.415049102157354]
[35, 200, 14.029502864927053]
[36, 200, 14.534267418086529]
[37

[276, 200, 163.1564825810492]
[277, 200, 163.840851534158]
[278, 200, 164.36948639899492]
[279, 200, 165.1023448817432]
[280, 200, 165.787160217762]
[281, 200, 166.46871137991548]
[282, 200, 167.04714464768767]
[283, 200, 167.74500216543674]
[284, 200, 168.37612803280354]
[285, 200, 169.15717351809144]
[286, 200, 169.70692430064082]
[287, 200, 170.4515691176057]
[288, 200, 171.08008040115237]
[289, 200, 171.8162242025137]
[290, 200, 172.38228625059128]
[291, 200, 173.11830543354154]
[292, 200, 173.73256599903107]
[293, 200, 174.43450728803873]
[294, 200, 175.03038453310728]
[295, 200, 175.76504101231694]
[296, 200, 176.3877769522369]
[297, 200, 177.12991078197956]
[298, 200, 177.6528933495283]
[299, 200, 178.38555723428726]
[300, 200, 179.07191678509116]
[301, 200, 179.8109517507255]
[302, 200, 180.32818394899368]
[303, 200, 181.0726052634418]
[304, 200, 181.6973763294518]
[305, 200, 182.44323340058327]
[306, 200, 183.02239498496056]
[307, 200, 183.73715993389487]
[308, 200, 184.417964

[545, 200, 350.93124245107174]
[546, 200, 351.5899532176554]
[547, 200, 352.32521548494697]
[548, 200, 353.02225971594453]
[549, 200, 353.8400243334472]
[550, 200, 354.49583265185356]
[551, 200, 355.2836530543864]
[552, 200, 356.01176380366087]
[553, 200, 356.7948293350637]
[554, 200, 357.38570276647806]
[555, 200, 358.23886663094163]
[556, 200, 358.9094224348664]
[557, 200, 359.6900451146066]
[558, 200, 360.3220274336636]
[559, 200, 361.1198925189674]
[560, 200, 361.8510499857366]
[561, 200, 362.6914247535169]
[562, 200, 363.27954242005944]
[563, 200, 364.0408998169005]
[564, 200, 364.78180005028844]
[565, 200, 365.57714883610606]
[566, 200, 366.1889613009989]
[567, 200, 367.02713026478887]
[568, 200, 367.71582105383277]
[569, 200, 368.4526848681271]
[570, 200, 369.1238401122391]
[571, 200, 369.8903124816716]
[572, 200, 370.60803316533566]
[573, 200, 371.4137075319886]
[574, 200, 372.03680542111397]
[575, 200, 372.86108550056815]
[576, 200, 373.56725941970944]
[577, 200, 374.337092749

[816, 200, 553.588692266494]
[817, 200, 554.4176734015346]
[818, 200, 555.0607360526919]
[819, 200, 555.9383602514863]
[820, 200, 556.7238066829741]
[821, 200, 557.5182987526059]
[822, 200, 558.1825708672404]
[823, 200, 558.982055015862]
[824, 200, 559.7135923504829]
[825, 200, 560.6077813357115]
[826, 200, 561.2617557160556]
[827, 200, 562.0590283349156]
[828, 200, 562.8177924007177]
[829, 200, 563.6162765361369]
[830, 200, 564.2713638320565]
[831, 200, 565.0901037491858]
[832, 200, 565.8043105639517]
[833, 200, 566.6452562324703]
[834, 200, 567.3219747133553]
[835, 200, 568.1385013870895]
[836, 200, 568.910048365593]
[837, 200, 569.7517300024629]
[838, 200, 570.3910707831383]
[839, 200, 571.1771837323904]
[840, 200, 572.0070804022253]
[841, 200, 572.8026998825371]
[842, 200, 573.4148678816855]
[843, 200, 574.2559204027057]
[844, 200, 574.9678421504796]
[845, 200, 575.8365348987281]
[846, 200, 576.5096940845251]
[847, 200, 577.3611364997923]
[848, 200, 578.061280850321]
[849, 200, 578

[1088, 200, 767.3963010683656]
[1089, 200, 768.2909963652492]
[1090, 200, 768.9947427511215]
[1091, 200, 769.8230711333454]
[1092, 200, 770.6542230509222]
[1093, 200, 771.4776354692876]
[1094, 200, 772.1215303502977]
[1095, 200, 773.0225973315537]
[1096, 200, 773.7658547312021]
[1097, 200, 774.6012787148356]
[1098, 200, 775.2971204482019]
[1099, 200, 776.1697961166501]
[1100, 200, 776.9951781332493]
[1101, 200, 777.8660036362708]
[1102, 200, 778.5254362821579]
[1103, 200, 779.3497456870973]
[1104, 200, 780.1415609009564]
[1105, 200, 781.0427191853523]
[1106, 200, 781.7409941144288]
[1107, 200, 782.6389071047306]
[1108, 200, 783.4069381020963]
[1109, 200, 784.2529462836683]
[1110, 200, 784.995759498328]
[1111, 200, 785.8604651987553]
[1112, 200, 786.6140493638813]
[1113, 200, 787.5278683193028]
[1114, 200, 788.1933205835521]
[1115, 200, 789.0932475812733]
[1116, 200, 789.9088051468134]
[1117, 200, 790.7646413818002]
[1118, 200, 791.4439085163176]
[1119, 200, 792.3036087527871]
[1120, 20

In [3]:
data=[]
import time
start=time.time()/60
for m in [1204..2403]:
    poly=H4(200,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/25sept22no2.txt','wb') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[1204, 200, 0.7923523187637329]
[1205, 200, 1.6813494861125946]
[1206, 200, 2.366381950676441]
[1207, 200, 3.2393316850066185]
[1208, 200, 3.99722708389163]
[1209, 200, 4.917757283896208]
[1210, 200, 5.632594618946314]
[1211, 200, 6.503068417310715]
[1212, 200, 7.313473518937826]
[1213, 200, 8.150695066899061]
[1214, 200, 8.822927683591843]
[1215, 200, 9.7435133010149]
[1216, 200, 10.509187284857035]
[1217, 200, 11.332779705524445]
[1218, 200, 12.080927465111017]
[1219, 200, 12.918236717581749]
[1220, 200, 13.715212769806385]
[1221, 200, 14.611278966069221]
[1222, 200, 15.277897216379642]
[1223, 200, 16.12558653205633]
[1224, 200, 16.952985119074583]
[1225, 200, 17.876931749284267]
[1226, 200, 18.52809651941061]
[1227, 200, 19.38734283298254]
[1228, 200, 20.1421060860157]
[1229, 200, 20.972431231290102]
[1230, 200, 21.720818366855383]
[1231, 200, 22.541507732123137]
[1232, 200, 23.36180728301406]
[1233, 200, 24.25117328390479]
[1234, 200, 24.943470250815153]
[1235, 200, 25.857061285525

[1465, 200, 215.70292698219419]
[1466, 200, 216.40096208453178]
[1467, 200, 217.29445076733828]
[1468, 200, 218.09783086553216]
[1469, 200, 218.97279405221343]
[1470, 200, 219.76253103092313]
[1471, 200, 220.6290975511074]
[1472, 200, 221.37735906615853]
[1473, 200, 222.27292161807418]
[1474, 200, 222.97398616746068]
[1475, 200, 223.89043763279915]
[1476, 200, 224.71423096582294]
[1477, 200, 225.6030932329595]
[1478, 200, 226.27095115184784]
[1479, 200, 227.18952311947942]
[1480, 200, 228.00767148286104]
[1481, 200, 228.86019380018115]
[1482, 200, 229.61549043655396]
[1483, 200, 230.483364533633]
[1484, 200, 231.31130518391728]
[1485, 200, 232.2829822152853]
[1486, 200, 232.97826255485415]
[1487, 200, 233.84843154996634]
[1488, 200, 234.66832296550274]
[1489, 200, 235.51134526729584]
[1490, 200, 236.21469279751182]
[1491, 200, 237.15715769678354]
[1492, 200, 237.9471725858748]
[1493, 200, 238.81713965162635]
[1494, 200, 239.53583331778646]
[1495, 200, 240.47988018020988]
[1496, 200, 24

[1726, 200, 433.0647944509983]
[1727, 200, 433.96403670310974]
[1728, 200, 434.7581253051758]
[1729, 200, 435.70314928516746]
[1730, 200, 436.4286535009742]
[1731, 200, 437.32976780086756]
[1732, 200, 438.13143768161535]
[1733, 200, 439.00486209988594]
[1734, 200, 439.7545736543834]
[1735, 200, 440.66882706806064]
[1736, 200, 441.50970516726375]
[1737, 200, 442.4310930520296]
[1738, 200, 443.16290716826916]
[1739, 200, 444.05885500088334]
[1740, 200, 444.9400027655065]
[1741, 200, 445.83688398450613]
[1742, 200, 446.55232805013657]
[1743, 200, 447.50539055094123]
[1744, 200, 448.27683666720986]
[1745, 200, 449.199131667614]
[1746, 200, 449.93943725153804]
[1747, 200, 450.8331122137606]
[1748, 200, 451.65563398599625]
[1749, 200, 452.6071868017316]
[1750, 200, 453.3490899465978]
[1751, 200, 454.2518211454153]
[1752, 200, 455.07055696472526]
[1753, 200, 455.94090908765793]
[1754, 200, 456.6450389325619]
[1755, 200, 457.6185704804957]
[1756, 200, 458.4057428650558]
[1757, 200, 459.3075211

[1991, 200, 658.0519022643566]
[1992, 200, 658.9086870513856]
[1993, 200, 659.7983123660088]
[1994, 200, 660.4901307485998]
[1995, 200, 661.5064396858215]
[1996, 200, 662.3081084676087]
[1997, 200, 663.1977000981569]
[1998, 200, 663.9505208842456]
[1999, 200, 664.849956586957]
[2000, 200, 665.6808352135122]
[2001, 200, 666.6331135854125]
[2002, 200, 667.3954492211342]
[2003, 200, 668.2644979171455]
[2004, 200, 669.1105368994176]
[2005, 200, 670.025124501437]
[2006, 200, 670.7455762177706]
[2007, 200, 671.6714156158268]
[2008, 200, 672.4938563816249]
[2009, 200, 673.4249898344278]
[2010, 200, 674.2204786688089]
[2011, 200, 675.0992927327752]
[2012, 200, 675.9127749986947]
[2013, 200, 676.8715540505946]
[2014, 200, 677.5841948017478]
[2015, 200, 678.5322110466659]
[2016, 200, 679.387389164418]
[2017, 200, 680.288052663207]
[2018, 200, 681.0042514838278]
[2019, 200, 681.9282203875482]
[2020, 200, 682.7598320841789]
[2021, 200, 683.6391845345497]
[2022, 200, 684.375330299139]
[2023, 200, 6

[2257, 200, 887.2570181488991]
[2258, 200, 887.9859167672694]
[2259, 200, 888.948886834085]
[2260, 200, 889.8216537497938]
[2261, 200, 890.794573970139]
[2262, 200, 891.5779658667743]
[2263, 200, 892.5037646330893]
[2264, 200, 893.326567016542]
[2265, 200, 894.3273313976824]
[2266, 200, 895.0783762820065]
[2267, 200, 896.0030945017934]
[2268, 200, 896.9146546684206]
[2269, 200, 897.8306402154267]
[2270, 200, 898.5915834009647]
[2271, 200, 899.5425716675818]
[2272, 200, 900.3340977542102]
[2273, 200, 901.2148717343807]
[2274, 200, 901.9923918135464]
[2275, 200, 902.9842288829386]
[2276, 200, 903.8328475356102]
[2277, 200, 904.805519066751]
[2278, 200, 905.5415647961199]
[2279, 200, 906.445155415684]
[2280, 200, 907.3579173982143]
[2281, 200, 908.2605083175004]
[2282, 200, 909.0209316834807]
[2283, 200, 909.9710523672402]
[2284, 200, 910.7905787676573]
[2285, 200, 911.7326757013798]
[2286, 200, 912.5067208372056]
[2287, 200, 913.4222918488085]
[2288, 200, 914.2746597826481]
[2289, 200, 9

In [4]:
data=[]
import time
start=time.time()/60
for m in [2404..3610]:
    poly=H4(200,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/26sept22no13.txt','wb') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[2404, 200, 0.849245298653841]
[2405, 200, 1.8086670823395252]
[2406, 200, 2.5741373486816883]
[2407, 200, 3.492508716881275]
[2408, 200, 4.34523643180728]
[2409, 200, 5.321867782622576]
[2410, 200, 6.0742916986346245]
[2411, 200, 7.001638233661652]
[2412, 200, 7.8979539312422276]
[2413, 200, 8.84703578427434]
[2414, 200, 9.588346164673567]
[2415, 200, 10.622328497469425]
[2416, 200, 11.44425966590643]
[2417, 200, 12.362289249897003]
[2418, 200, 13.16026484966278]
[2419, 200, 14.07607926428318]
[2420, 200, 14.974933296442032]
[2421, 200, 15.927364353090525]
[2422, 200, 16.692532148212194]
[2423, 200, 17.59969114884734]
[2424, 200, 18.466950349509716]
[2425, 200, 19.409626584500074]
[2426, 200, 20.13689448311925]
[2427, 200, 21.084535717964172]
[2428, 200, 21.91056028008461]
[2429, 200, 22.854342617094517]
[2430, 200, 23.66002554818988]
[2431, 200, 24.614212214946747]
[2432, 200, 25.409742031246424]
[2433, 200, 26.378067050129175]
[2434, 200, 27.106990348547697]
[2435, 200, 28.058228883

[2665, 200, 230.7695867791772]
[2666, 200, 231.51976466923952]
[2667, 200, 232.502550881356]
[2668, 200, 233.3585962317884]
[2669, 200, 234.27304903045297]
[2670, 200, 235.09074186906219]
[2671, 200, 235.98531726375222]
[2672, 200, 236.8102068528533]
[2673, 200, 237.77534079924226]
[2674, 200, 238.55250791460276]
[2675, 200, 239.4980115480721]
[2676, 200, 240.3804624862969]
[2677, 200, 241.2871547676623]
[2678, 200, 242.03745314851403]
[2679, 200, 243.0149196870625]
[2680, 200, 243.89144598320127]
[2681, 200, 244.84023363515735]
[2682, 200, 245.62609591707587]
[2683, 200, 246.56827406212687]
[2684, 200, 247.43386099860072]
[2685, 200, 248.44245168194175]
[2686, 200, 249.20532768592238]
[2687, 200, 250.12536326795816]
[2688, 200, 250.99965520203114]
[2689, 200, 251.9099355340004]
[2690, 200, 252.68938445299864]
[2691, 200, 253.67664408311248]
[2692, 200, 254.51286408305168]
[2693, 200, 255.43409176543355]
[2694, 200, 256.19987685233355]
[2695, 200, 257.2023281827569]
[2696, 200, 258.029

[2926, 200, 462.0082548968494]
[2927, 200, 462.91877980157733]
[2928, 200, 463.7939264960587]
[2929, 200, 464.7245545499027]
[2930, 200, 465.51157515123487]
[2931, 200, 466.4750197157264]
[2932, 200, 467.3207003362477]
[2933, 200, 468.29867384955287]
[2934, 200, 469.0778127834201]
[2935, 200, 470.06052416935563]
[2936, 200, 470.90503191947937]
[2937, 200, 471.89164987951517]
[2938, 200, 472.6518792808056]
[2939, 200, 473.58334260061383]
[2940, 200, 474.54221668094397]
[2941, 200, 475.47573816776276]
[2942, 200, 476.23180359974504]
[2943, 200, 477.19333143532276]
[2944, 200, 478.0070566497743]
[2945, 200, 478.99408020079136]
[2946, 200, 479.7624863013625]
[2947, 200, 480.70459693297744]
[2948, 200, 481.5527126304805]
[2949, 200, 482.5335652641952]
[2950, 200, 483.3114470615983]
[2951, 200, 484.2687244154513]
[2952, 200, 485.158927667886]
[2953, 200, 486.0955003798008]
[2954, 200, 486.84477504715323]
[2955, 200, 487.8629874177277]
[2956, 200, 488.7130246683955]
[2957, 200, 489.6542120836

[3191, 200, 699.2118433825672]
[3192, 200, 700.1446663998067]
[3193, 200, 701.0857751518488]
[3194, 200, 701.8432060666382]
[3195, 200, 702.8599418140948]
[3196, 200, 703.7388005666435]
[3197, 200, 704.6657297834754]
[3198, 200, 705.4687363132834]
[3199, 200, 706.436061900109]
[3200, 200, 707.2817166000605]
[3201, 200, 708.2996069863439]
[3202, 200, 709.0468952842057]
[3203, 200, 709.9946780316532]
[3204, 200, 710.86021098122]
[3205, 200, 711.8421958647668]
[3206, 200, 712.6176062338054]
[3207, 200, 713.5960480161011]
[3208, 200, 714.4358848147094]
[3209, 200, 715.3640029355884]
[3210, 200, 716.2018243670464]
[3211, 200, 717.169702950865]
[3212, 200, 718.0749640017748]
[3213, 200, 719.0962925329804]
[3214, 200, 719.8590757995844]
[3215, 200, 720.8142739310861]
[3216, 200, 721.7108955346048]
[3217, 200, 722.6452975012362]
[3218, 200, 723.381087332964]
[3219, 200, 724.3741962499917]
[3220, 200, 725.3163417167962]
[3221, 200, 726.2521047331393]
[3222, 200, 727.0437475144863]
[3223, 200, 7

[3457, 200, 940.0502935312688]
[3458, 200, 940.8709484152496]
[3459, 200, 941.866888333112]
[3460, 200, 942.7579483166337]
[3461, 200, 943.7044407837093]
[3462, 200, 944.5012955963612]
[3463, 200, 945.4538422822952]
[3464, 200, 946.333357449621]
[3465, 200, 947.4016938991845]
[3466, 200, 948.1801747493446]
[3467, 200, 949.1297595985234]
[3468, 200, 950.0583859682083]
[3469, 200, 951.0030511803925]
[3470, 200, 951.8056090511382]
[3471, 200, 952.7983008176088]
[3472, 200, 953.6479681320488]
[3473, 200, 954.6069573499262]
[3474, 200, 955.4152955785394]
[3475, 200, 956.4214630834758]
[3476, 200, 957.3415851183236]
[3477, 200, 958.3338048830628]
[3478, 200, 959.1043829135597]
[3479, 200, 960.0626485347748]
[3480, 200, 961.0189166478813]
[3481, 200, 961.9429269507527]
[3482, 200, 962.7129529155791]
[3483, 200, 963.7003353871405]
[3484, 200, 964.6056633330882]
[3485, 200, 965.5929223485291]
[3486, 200, 966.4256220497191]
[3487, 200, 967.3796138316393]
[3488, 200, 968.22110016644]
[3489, 200, 

In [5]:
data=[]
import time
start=time.time()/60
for m in [3611..4818]:
    poly=H4(200,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/27sept22no1.txt','wb') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[3611, 200, 1.0966309681534767]
[3612, 200, 2.1496217362582684]
[3613, 200, 3.1424203366041183]
[3614, 200, 3.9676333367824554]
[3615, 200, 5.041922736912966]
[3616, 200, 5.9170026034116745]
[3617, 200, 6.917526949197054]
[3618, 200, 7.755414169281721]
[3619, 200, 8.786965288221836]
[3620, 200, 9.7019326351583]
[3621, 200, 10.738296136260033]
[3622, 200, 11.513543035835028]
[3623, 200, 12.466360084712505]
[3624, 200, 13.365671653300524]
[3625, 200, 14.367694336920977]
[3626, 200, 15.189440622925758]
[3627, 200, 16.207194849848747]
[3628, 200, 17.09548083692789]
[3629, 200, 18.05790165066719]
[3630, 200, 18.904691137373447]
[3631, 200, 19.860571902245283]
[3632, 200, 20.71355013549328]
[3633, 200, 21.7425905354321]
[3634, 200, 22.514046002179384]
[3635, 200, 23.51742808520794]
[3636, 200, 24.433894600719213]
[3637, 200, 25.404809050261974]
[3638, 200, 26.194538816809654]
[3639, 200, 27.207314651459455]
[3640, 200, 28.142634484916925]
[3641, 200, 29.110726919025183]
[3642, 200, 29.917233

[3872, 200, 241.76713316887617]
[3873, 200, 242.74175510182977]
[3874, 200, 243.5196149162948]
[3875, 200, 244.49628683552146]
[3876, 200, 245.45056047290564]
[3877, 200, 246.4889287352562]
[3878, 200, 247.41835376992822]
[3879, 200, 248.5621142052114]
[3880, 200, 249.63153228536248]
[3881, 200, 250.73457735404372]
[3882, 200, 251.6754359714687]
[3883, 200, 252.79732130095363]
[3884, 200, 253.80095770210028]
[3885, 200, 255.04213676974177]
[3886, 200, 255.95807626843452]
[3887, 200, 256.99973575398326]
[3888, 200, 257.9141270853579]
[3889, 200, 258.8856843672693]
[3890, 200, 259.6669698357582]
[3891, 200, 260.6683074682951]
[3892, 200, 261.5749322511256]
[3893, 200, 262.53805488720536]
[3894, 200, 263.36539160460234]
[3895, 200, 264.3851851709187]
[3896, 200, 265.2666720151901]
[3897, 200, 266.26059091463685]
[3898, 200, 267.0422013550997]
[3899, 200, 268.0219008848071]
[3900, 200, 269.00944113358855]
[3901, 200, 269.967159319669]
[3902, 200, 270.7406540699303]
[3903, 200, 271.73519805

[4133, 200, 484.65148751810193]
[4134, 200, 485.50300446897745]
[4135, 200, 486.48947255313396]
[4136, 200, 487.4014156870544]
[4137, 200, 488.4364365041256]
[4138, 200, 489.2181183844805]
[4139, 200, 490.20826468244195]
[4140, 200, 491.1958525031805]
[4141, 200, 492.1988596357405]
[4142, 200, 492.9956467524171]
[4143, 200, 494.0187310874462]
[4144, 200, 494.91016525030136]
[4145, 200, 495.9073358178139]
[4146, 200, 496.7114916704595]
[4147, 200, 497.7101748883724]
[4148, 200, 498.6202527657151]
[4149, 200, 499.6309794001281]
[4150, 200, 500.46231753751636]
[4151, 200, 501.4572705999017]
[4152, 200, 502.3797378875315]
[4153, 200, 503.35425263270736]
[4154, 200, 504.15103225409985]
[4155, 200, 505.1967205181718]
[4156, 200, 506.07602970302105]
[4157, 200, 507.05207650363445]
[4158, 200, 507.9224792532623]
[4159, 200, 508.89399360120296]
[4160, 200, 509.7898079864681]
[4161, 200, 510.80886390432715]
[4162, 200, 511.5872474834323]
[4163, 200, 512.5535507500172]
[4164, 200, 513.48326505348

[4398, 200, 731.1678107865155]
[4399, 200, 732.151106454432]
[4400, 200, 733.0888066701591]
[4401, 200, 734.0987072177231]
[4402, 200, 734.8787178210914]
[4403, 200, 735.8879738226533]
[4404, 200, 736.8076417855918]
[4405, 200, 737.8114505186677]
[4406, 200, 738.5633624531329]
[4407, 200, 739.5905998013914]
[4408, 200, 740.4879900664091]
[4409, 200, 741.4800994545221]
[4410, 200, 742.3614618368447]
[4411, 200, 743.3694245181978]
[4412, 200, 744.2494990155101]
[4413, 200, 745.2599308863282]
[4414, 200, 746.0338776856661]
[4415, 200, 747.0389718711376]
[4416, 200, 747.946858920157]
[4417, 200, 748.9500940181315]
[4418, 200, 749.7504336535931]
[4419, 200, 750.7466466017067]
[4420, 200, 751.7100677192211]
[4421, 200, 752.6744675040245]
[4422, 200, 753.5177778713405]
[4423, 200, 754.4753810353577]
[4424, 200, 755.3908861204982]
[4425, 200, 756.4454164505005]
[4426, 200, 757.2285406701267]
[4427, 200, 758.222964450717]
[4428, 200, 759.1638484708965]
[4429, 200, 760.1408287845552]
[4430, 200,

[4664, 200, 979.761662401259]
[4665, 200, 980.8208198994398]
[4666, 200, 981.6086370199919]
[4667, 200, 982.6001220867038]
[4668, 200, 983.5498389527202]
[4669, 200, 984.5684847049415]
[4670, 200, 985.406665686518]
[4671, 200, 986.4105179533362]
[4672, 200, 987.2824204862118]
[4673, 200, 988.2360640540719]
[4674, 200, 989.0863086692989]
[4675, 200, 990.1279900856316]
[4676, 200, 991.0814233534038]
[4677, 200, 992.0976548716426]
[4678, 200, 992.8795392513275]
[4679, 200, 993.8624765016139]
[4680, 200, 994.8510354012251]
[4681, 200, 995.8423422500491]
[4682, 200, 996.6203175000846]
[4683, 200, 997.6739386506379]
[4684, 200, 998.5818776674569]
[4685, 200, 999.5849532075226]
[4686, 200, 1000.449248470366]
[4687, 200, 1001.4195661507547]
[4688, 200, 1002.3119442723691]
[4689, 200, 1003.3319761194289]
[4690, 200, 1004.1976109519601]
[4691, 200, 1005.1758768856525]
[4692, 200, 1006.1523576565087]
[4693, 200, 1007.1646172180772]
[4694, 200, 1007.9349523037672]
[4695, 200, 1008.9899845682085]
[